In [1]:
from Hamil_search import *
from simulation import *
import numpy as np
import os
import pennylane as qml

In [ ]:
def test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc):
    A = Henc @ HpenInverse @ Henc
    off_diag = (P0 - Penc) @ A @ Penc
    if checkSame(off_diag, np.zeros(off_diag.shape)):
        print("no leakage")
    else:
        print("yes leakage")
    Hlogi = - Uenc.conj().T @ A @ Uenc
    return qml.pauli_decompose(Hlogi, hide_identity=True)


def get_P0_HpenInverse(Hpen):
    """
    Make sure that the eigenvalues of Hpen are all integers
    """
    e, u = np.linalg.eigh(Hpen)

    a = np.array([1 if abs(x) < 0.5 else 0 for x in e])
    P0 = u @ (np.expand_dims(a, axis=1) * u.conj().T)

    b = np.array([1 / x if abs(x) > 0.5 else 0 for x in e])
    HpenInverse = u @ (np.expand_dims(b, axis=1) * u.conj().T)

    return P0, HpenInverse

In [ ]:
n_block = 2
n_phys = 4 * n_block
dim_phys = 2 ** n_phys


def idx(b, i):
    """
    Return the index of the physical qubit at the `i`-th position of the `b`-th block.
    `b` and the returned index start from 0.
    `i` takes value from {1,2,3,4}.
    """
    assert 0 <= b and b < n_block
    assert 1 <= i and i <= 4
    return b * 4 + i - 1


gx, gz = 1, 3
Hpen_terms = []
for b in range(n_block):
    Hpen_terms += [(gx, f'X{idx(b, 1)}*X{idx(b, 2)}'), (gz, f'Z{idx(b, 1)}*Z{idx(b, 2)}'),
                   (gx, f'X{idx(b, 3)}*X{idx(b, 4)}'), (gz, f'Z{idx(b, 3)}*Z{idx(b, 4)}')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

noise_strength = 1.0
np.random.seed(42)
V_terms = []
eps_x = noise_strength * np.random.uniform(-1, 1, n_phys)
eps_z = noise_strength * np.random.uniform(-1, 1, n_phys)
eps_x = noise_strength * np.random.uniform(-1, 1, n_phys)
V_terms += [(1, f'Z{i}') for i in range(n_phys)]
V_terms += [(1, f'X{i}') for i in range(n_phys)]
V = blocks2Mat(n_phys, V_terms)

# $ZZ+X$

In [58]:
Henc1_terms = [(1, f'Z{idx(0, 2)}*Z{idx(0, 3)}'), (1, f'Z{idx(1, 2)}*Z{idx(1, 3)}')]
Henc1_terms += [(-1, f'X{idx(0, 1)}*X{idx(0, 3)}'), (1, f'X{idx(0, 1)}*X{idx(0, 2)}')]
Henc1_terms += [(-1, f'X{idx(1, 1)}*X{idx(1, 3)}'), (1, f'X{idx(1, 1)}*X{idx(1, 2)}')]
Henc1_terms += [(1, f'Z{idx(1, 1)}*Z{idx(1, 2)}')]
Henc1 = blocks2Mat(n_phys, Henc1_terms)

Henc2_terms = [(1, f'Z{idx(0, 2)}*X{idx(1, 3)}'), (1, f'Z{idx(0, 4)}*X{idx(1, 3)}')]
Henc2 = blocks2Mat(n_phys, Henc2_terms)
Henc2 = np.sqrt(8 / 3) * Henc2

In [59]:
logi2 = test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc2)

no leakage


In [60]:
logi1 = qml.pauli_decompose(Uenc.conj().T @ Henc1 @ Uenc, hide_identity=True)

In [61]:
logi = qml.simplify(logi1 + logi2)
logi

(
    0.9999999999999996 * X(3)
  + 0.9999999999999996 * X(2)
  + -2.220446049250313e-16 * Z(2)
  + 0.9999999999999996 * (Z(2) @ Z(3))
  + 0.9999999999999996 * X(1)
  + 0.9999999999999996 * X(0)
  + 0.9999999999999996 * (Z(0) @ Z(1))
  + 0.9999999999999998 * (Z(1) @ Z(2))
)

In [62]:
# Make sure P0 @ Henc2 @ P0 == 0
np.allclose(P0 @ Henc2 @ P0, 0)

True

In [63]:
Heff3_part1 = P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0
np.allclose(Heff3_part1, 0)

True

In [64]:
Heff3_part2 = P0 @ (Henc1 + V) @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ (Henc1 + V) @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ (Henc1 + V) @ P0 \
    - 0.5 * (P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0 @ (Henc1 + V) @ P0 +
             P0 @ (Henc1 + V) @ P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0)
np.allclose(Heff3_part2, 0)

False

In [65]:
Heff3 = Heff3_part1 + Heff3_part2
np.allclose((P0 - Penc) @ Heff3 @ Penc, 0)

True

In [66]:
np.linalg.norm((P0 - Penc) @ Heff3 @ Penc, ord=None)

np.float64(1.993068199857521e-14)

In [67]:
Heff3_block_diagonal = Penc @ Heff3 @ Penc + (P0-Penc) @ Heff3 @ (P0-Penc)
Heff3_block_off_diagonal = (P0-Penc) @ Heff3 @ Penc + Penc @ Heff3 @ (P0-Penc)
assert np.allclose(Heff3_block_diagonal + Heff3_block_off_diagonal, Heff3)
norm_block_off_diagonal = np.linalg.norm(Heff3_block_off_diagonal, ord=2)
norm = np.linalg.norm(Heff3, ord=2)
norm_block_off_diagonal / norm

np.float64(6.866576786931088e-15)

In [69]:
3 * 8 * qml.pauli_decompose(Uenc.conj().T @ Heff3_part2 @ Uenc, hide_identity=True)

(
    -8.999999999999996 * X(2)
  + -3.0000000000000053 * Z(2)
  + -10.000000000000002 * (Z(2) @ Z(3))
  + -5.0 * X(1)
  + 8.999999999999993 * (Z(1) @ X(2))
  + 3.000000000000012 * (Z(1) @ Z(2))
  + 9.999999999999995 * (Z(1) @ Z(2) @ Z(3))
  + 0.9999999999999822 * Z(0)
  + -0.9999999999999756 * (Z(0) @ Z(1))
)

# $XX+Z$

In [39]:
Henc1_terms = [(-1, f'X{idx(0, 2)}*X{idx(0, 3)}'), (-1, f'X{idx(1, 2)}*X{idx(1, 3)}')]
Henc1_terms += [(1, f'Z{idx(0, 1)}*Z{idx(0, 2)}'), (1, f'Z{idx(0, 1)}*Z{idx(0, 3)}')]
Henc1_terms += [(1, f'Z{idx(1, 1)}*Z{idx(1, 2)}'), (1, f'Z{idx(1, 1)}*Z{idx(1, 3)}')]
Henc1_terms += [(1, f'X{idx(0, 1)}*X{idx(0, 2)}')]
Henc1 = blocks2Mat(n_phys, Henc1_terms)

Henc2_terms = [(1, f'Z{idx(0, 2)}*X{idx(1, 1)}'), (1, f'Z{idx(0, 2)}*X{idx(1, 3)}')]
Henc2 = blocks2Mat(n_phys, Henc2_terms)
Henc2 = np.sqrt(8) * Henc2

In [40]:
logi2 = test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc2)

no leakage


In [41]:
logi1 = qml.pauli_decompose(Uenc.conj().T @ Henc1 @ Uenc, hide_identity=True)

In [42]:
logi = qml.simplify(logi1 + logi2)
logi

(
    0.9999999999999996 * Z(3)
  + 0.9999999999999996 * (X(2) @ X(3))
  + 0.9999999999999996 * Z(2)
  + -4.440892098500626e-16 * X(1)
  + 0.9999999999999996 * Z(1)
  + 0.9999999999999996 * (X(0) @ X(1))
  + 0.9999999999999996 * Z(0)
  + 1.0 * (X(1) @ X(2))
)

In [43]:
# Make sure P0 @ Henc2 @ P0 == 0
np.allclose(P0 @ Henc2 @ P0, 0)

True

In [56]:
Heff3_part1 = P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0
np.allclose(Heff3_part1, 0)
np.linalg.norm(P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0, ord=2)

np.float64(5.747860582248459e-15)

In [52]:
Q0 = np.eye(P0.shape[0]) - P0

In [57]:
np.linalg.norm(P0 @ Henc2 @ HpenInverse, ord=2)


np.float64(1.1180339887498965)

In [45]:
Heff3_part2 = P0 @ (Henc1 + V) @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ (Henc1 + V) @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ (Henc1 + V) @ P0 \
    - 0.5 * (P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0 @ (Henc1 + V) @ P0 +
             P0 @ (Henc1 + V) @ P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0)
np.allclose(Heff3_part2, 0)

False

In [46]:
Heff3 = Heff3_part1 + Heff3_part2
np.linalg.norm((P0 - Penc) @ Heff3 @ Penc, ord=None)

np.float64(4.822000022158139e-14)

In [48]:
8 * qml.pauli_decompose(Uenc.conj().T @ Heff3_part2 @ Uenc, hide_identity=True)

(
    9.00000000000001 * X(3)
  + -10.0 * Z(3)
  + -9.000000000000004 * (X(2) @ X(3))
  + 10.000000000000002 * (X(2) @ Z(3))
  + -5.000000000000007 * Z(2)
  + -7.000000000000007 * X(1)
  + 7.000000000000001 * (X(1) @ X(2))
  + -0.9999999999999978 * Z(1)
  + 0.9999999999999951 * (Z(1) @ X(2))
  + -10.000000000000009 * (X(0) @ X(1))
  + 10.000000000000014 * (X(0) @ X(1) @ X(2))
)

In [51]:
Q0 = np.eye(P0.shape[0]) - P0
np.linalg.norm(Q0 @ Henc2 @ Q0, ord=None)

np.float64(54.25863986500215)

# $XX+ZZ+X+Z$

In [29]:
# Inner-block ZZ and XX
Henc1_terms = [(1, f'Z{idx(0, 2)}*Z{idx(0, 3)}'), (1, f'Z{idx(1, 2)}*Z{idx(1, 3)}')]
Henc1_terms += [(-1, f'X{idx(0, 2)}*X{idx(0, 3)}'), (-1, f'X{idx(1, 2)}*X{idx(1, 3)}')]
# Single-qubit logical Z and X
Henc1_terms += [(1, f'Z{idx(0, 1)}*Z{idx(0, 2)}'), (1, f'Z{idx(0, 1)}*Z{idx(0, 3)}')]
Henc1_terms += [(1, f'Z{idx(1, 1)}*Z{idx(1, 2)}'), (1, f'Z{idx(1, 1)}*Z{idx(1, 3)}')]
Henc1_terms += [(-1, f'X{idx(0, 1)}*X{idx(0, 3)}'), (1, f'X{idx(0, 1)}*X{idx(0, 2)}')]
Henc1_terms += [(-1, f'X{idx(1, 1)}*X{idx(1, 3)}'), (1, f'X{idx(1, 1)}*X{idx(1, 2)}')]
# Compensation term # TODO
Henc1_terms += [(1.5, f'X{idx(0, 1)}*X{idx(0, 2)}'), (-3.5, f'Z{idx(1, 1)}*Z{idx(1, 2)}')]
Henc1 = blocks2Mat(n_phys, Henc1_terms)
# Cross-block ZZ+XX
Henc2_terms = [(1, f'Z{idx(0, 2)}*X{idx(1, 3)}'), (1, f'Z{idx(0, 4)}*X{idx(1, 3)}'), (3, f'Z{idx(0, 2)}*X{idx(1, 1)}')]
Henc2 = blocks2Mat(n_phys, Henc2_terms)
Henc2 = np.sqrt(8 / 3) * Henc2

In [30]:
logi2 = test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc2)

no leakage


In [31]:
logi1 = qml.pauli_decompose(Uenc.conj().T @ Henc1 @ Uenc, hide_identity=True)

In [32]:
logi = qml.simplify(logi1 + logi2)
logi

(
    0.9999999999999996 * X(3)
  + 0.9999999999999996 * Z(3)
  + 0.9999999999999996 * X(2)
  + 0.9999999999999996 * (X(2) @ X(3))
  + 0.9999999999999996 * Z(2)
  + 0.9999999999999996 * (Z(2) @ Z(3))
  + 0.9999999999999996 * X(1)
  + 0.9999999999999996 * Z(1)
  + 0.9999999999999996 * X(0)
  + 0.9999999999999996 * (X(0) @ X(1))
  + 0.9999999999999996 * Z(0)
  + 0.9999999999999996 * (Z(0) @ Z(1))
  + 0.9999999999999997 * (X(1) @ X(2))
  + 0.9999999999999998 * (Z(1) @ Z(2))
)

In [33]:
# Make sure P0 @ Henc2 @ P0 == 0
np.allclose(P0 @ Henc2 @ P0, 0)

True

In [34]:
Heff3_part1 = P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0
np.allclose(Heff3_part1, 0)

True

In [35]:
Heff3_part2 = P0 @ (Henc1 + V) @ HpenInverse @ Henc2 @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ (Henc1 + V) @ HpenInverse @ Henc2 @ P0 \
    + P0 @ Henc2 @ HpenInverse @ Henc2 @ HpenInverse @ (Henc1 + V) @ P0 \
    - 0.5 * (P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0 @ (Henc1 + V) @ P0 +
             P0 @ (Henc1 + V) @ P0 @ Henc2 @ HpenInverse @ HpenInverse @ Henc2 @ P0)
np.allclose(Heff3_part2, 0)

False

In [36]:
Heff3 = Heff3_part1 + Heff3_part2
np.linalg.norm((P0 - Penc) @ Heff3 @ Penc, ord=None)

np.float64(1.6499944768020453e-13)

In [37]:
qml.pauli_decompose(Uenc.conj().T @ Heff3_part2 @ Uenc, hide_identity=True)

(
    1.166666666666667 * (I(0) @ I(1) @ I(2) @ I(3))
  + 1.1250000000000042 * X(3)
  + -2.2916666666666647 * Z(3)
  + -2.1874999999999987 * X(2)
  + -2.0625000000000018 * (X(2) @ X(3))
  + 1.2499999999999993 * (X(2) @ Z(3))
  + 7.187499999999999 * Z(2)
  + -0.4166666666666723 * (Z(2) @ Z(3))
  + -8.020833333333334 * X(1)
  + 4.0625 * (X(1) @ X(2))
  + 0.375000000000002 * (X(1) @ X(2) @ Z(3))
  + 1.124999999999998 * (X(1) @ Z(2) @ Z(3))
  + 0.7499999999999958 * (Y(1) @ Y(2) @ Z(3))
  + -1.354166666666666 * Z(1)
  + -1.1249999999999991 * (Z(1) @ X(3))
  + 0.4166666666666683 * (Z(1) @ Z(3))
  + 0.4999999999999966 * (Z(1) @ X(2))
  + 0.3750000000000031 * (Z(1) @ X(2) @ X(3))
  + -1.2499999999999987 * (Z(1) @ X(2) @ Z(3))
  + 0.31250000000000044 * (Z(1) @ Z(2))
  + 0.4166666666666696 * (Z(1) @ Z(2) @ Z(3))
  + 1.0000000000000058 * (X(0) @ Z(2))
  + -2.0833333333333375 * (X(0) @ X(1))
  + 1.2500000000000004 * (X(0) @ X(1) @ X(2))
  + 0.12500000000000067 * (X(0) @ Z(1) @ Z(2))
  + 0.04166666